In [ ]:
# %% Deep learning - Section 11.114
#    Shifted MNIST

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

# Load data
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Split labels from data
labels = data[:,0]
data   = data[:,1:]

# Normalise data (original range is (0,255))
data_norm = data / np.max(data)


In [ ]:
# %% A note

# We know from the previous video that shifting an image (or scrambling it)
# will not affect the classification, if we do it consistently and for both
# train and test data.
# Here we will leave the train data as they are, and only shift ('roll') the
# test data by a few pixels


In [ ]:
# %% Create train and test datasets

# Convert to tensor (float and integers) the scrambled data
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% How to shift a vectorized image

# Grab one image and reshape to 2D
tmp = test_loader.dataset.tensors[0][0,:]
tmp = tmp.reshape(28,28)

# Shift the image (pytorch calls it "rolling"), dim=0 for vertical shift
tmpS = torch.roll(tmp,8,dims=1)

# Plot
phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*6*phi,6))

ax[0].imshow(tmp, cmap='gray')
ax[0].set_title('Original')

ax[1].imshow(tmpS, cmap='gray')
ax[1].set_title('Shifted (rolled)')

plt.savefig('figure60_shifted_mnist.png')

plt.show()

files.download('figure60_shifted_mnist.png')


In [ ]:
# %% Now shift all images in the test set
# Note: run the previous cell again to confirm the shifting

for i in range(test_loader.dataset.tensors[0].shape[0]):

  # Get the image
  img = test_loader.dataset.tensors[0][i,:]

  # Reshape and roll by max. 10 pixels, dim=0 for vertical shift
  randroll = np.random.randint(-10,11)
  img      = torch.roll( img.reshape(28,28),randroll,dims=1 )

  # Re-vectorize and put back into the matrix
  test_loader.dataset.tensors[0][i,:] = img.reshape(1,-1)


In [ ]:
# %% Function to generate the model

def gen_model():

    class mnist_FFN(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(784,64)
            self.fc1    = nn.Linear( 64,32)
            self.fc2    = nn.Linear( 32,32)
            self.output = nn.Linear( 32,10)

        # Forward propagation
        def forward(self,x):

            x = F.relu(self.input(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.output(x)

            return x

    # Create model instance
    ANN = mnist_FFN()

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer (SGD to slow down learning for illustration purpose)
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [ ]:
# %% Function to train the model

def train_model():

    # Parameters, model instance, inizialise vars
    num_epochs = 60
    ANN,loss_fun,optimizer = gen_model()

    losses_trn = []
    losses_tst = []
    train_acc  = []
    test_acc   = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses_trn.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
        test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )
        loss = loss_fun(yHat,y)
        losses_tst.append(loss.item())

        ANN.train()

    return train_acc,test_acc,losses_trn,losses_tst,ANN


In [ ]:
# %% Fit the model

train_acc,test_acc,losses_trn,losses_tst,ANN = train_model()


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*6*phi,6))

ax[0].plot(losses_trn,label='Train loss')
ax[0].plot(losses_tst,label='Test loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_ylim([0,3])
ax[0].set_title('Model loss')
ax[0].legend()

ax[1].plot(train_acc,label='Train accuracy')
ax[1].plot(test_acc,label='Test accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_ylim([10,100])
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%\nNormalised train and test data')
ax[1].legend()

plt.savefig('figure62_shifted_mnist.png')

plt.show()

files.download('figure62_shifted_mnist.png')


In [ ]:
# %% Exercise 1
#    You saw that shifting by max-10 pixels has negative effects on model generalization. What do you think will happen
#    if you shift the images by 4 pixels? 3? 2? 1? Try it and find out!

# The hypothesis is that if the logics hold, then no matter the amount of
# shifting, the performance should decrease; however, it's true that the digits
# have a certain "thikness", and that for example shifting by 1 or 2 pixels
# might not compromise the performace as much as a shifting that is larger that
# that the average width of a digit (what I mean is that if you overlap an
# original image to an image shifted by one pixel, the ovelap will still look
# a lot like to the original image).
#
# And OMG, the prediction was accurate ! Almost can't belive it.

# Reset loaders
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

# Shift all images in the test set by a fixed amount
for i in range(test_loader.dataset.tensors[0].shape[0]):

  img = test_loader.dataset.tensors[0][i,:]
  n   = 1
  img = torch.roll( img.reshape(28,28),n,dims=1 )
  test_loader.dataset.tensors[0][i,:] = img.reshape(1,-1)


In [ ]:
# %% Exercise 2
#    Dropout regularization helps make representations more distributed. Can that help with the decrease in learning?
#    Try this by testing the model on 2-pixel shifted images with and without 50% dropout.

# Surprisingly and yet not surprisingly, dropout doesn't help. The training
# accuracy goes down but that expected, the test accuracy however, stays the
# same. This is kind of expected because only the test data are shifted

# Reset loaders
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

# Shift all images in the test set by a fixed amount
for i in range(test_loader.dataset.tensors[0].shape[0]):

  img = test_loader.dataset.tensors[0][i,:]
  n   = 2
  img = torch.roll( img.reshape(28,28),n,dims=1 )
  test_loader.dataset.tensors[0][i,:] = img.reshape(1,-1)

# Function to generate the model
def gen_model(drop_rate):

    class mnist_FFN(nn.Module):
        def __init__(self,dropout_rate):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(784,64)
            self.fc1    = nn.Linear( 64,32)
            self.fc2    = nn.Linear( 32,32)
            self.output = nn.Linear( 32,10)

            # Dropout
            self.dr = dropout_rate

        # Forward propagation
        def forward(self,x):

            x = F.relu(self.input(x))
            x = F.dropout(x,p=self.dr,training=self.training)
            x = F.relu(self.fc1(x))
            x = F.dropout(x,p=self.dr,training=self.training)
            x = F.relu(self.fc2(x))
            x = F.dropout(x,p=self.dr,training=self.training)

            x = self.output(x)

            return x

    # Create model instance
    ANN = mnist_FFN(drop_rate)

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer

# Function to train the model
def train_model(dropout_rate):

    # Parameters, model instance, inizialise vars
    num_epochs = 60
    ANN,loss_fun,optimizer = gen_model(dropout_rate)

    losses_trn = []
    losses_tst = []
    train_acc  = []
    test_acc   = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses_trn.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
        test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )
        loss = loss_fun(yHat,y)
        losses_tst.append(loss.item())

        ANN.train()

    return train_acc,test_acc,losses_trn,losses_tst,ANN

# Fit the model
drop_rate = 0
train_acc,test_acc,losses_trn,losses_tst,ANN = train_model(drop_rate)


In [ ]:
# %% Exercise 3
#    Continue exploring the torch.roll() function. Make sure you know how to shift up and down, not just left/right.

# For that it's just enough to change the dims parameter to 0. And just for
# reference, the results are the same.

# Reset loaders
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

# Shift all images in the test set, dim=0 for vertical shift
for i in range(test_loader.dataset.tensors[0].shape[0]):

  img = test_loader.dataset.tensors[0][i,:]
  n   = np.random.randint(-10,11)
  img = torch.roll( img.reshape(28,28),n,dims=0 )
  test_loader.dataset.tensors[0][i,:] = img.reshape(1,-1)
